In [3]:
from copy import deepcopy

import pandas as pd

from BitBoard import osero
from osero_learn import learn

PLAY_WAY = deepcopy(osero.PLAY_WAY)
del PLAY_WAY["human"]
PLAY_WAY = PLAY_WAY.values()

In [4]:
eva = [[
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0
] for i in range(2)]

In [5]:
df = pd.DataFrame({})

check_point = [i for i in range(5, 64, 5)]

for i in range(10):
    print("\r[" + "#" * (i+1) + " " * (10-i+1) + "]", end="")
    for black in PLAY_WAY:
        for white in PLAY_WAY:
            run = learn(\
                black,
                white,
                check_point=check_point,
                seed_num=i,
                eva=eva
            )
            data = run.play()
            df = df.append(data, ignore_index=True)

print("\r[" + "#" * 10 + "]")

[##########]


In [20]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

x_data = df.drop(["turn", "last_score"], axis=1)
y_data = df[["turn_num", "last_score"]]

turn_vari = df["turn_num"].unique()

In [18]:
def two_plot(x, y, xlabel, ylabel, title, save_dir):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(x, y[0], label="train score")
    plt.plot(x, y[1], label="test score")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.savefig(save_dir + "/" + title)
    plt.clf()
    plt.close()

# def two_bar(x, x_name, y, xlabel, ylabel, title, save_dir):
#     fig = plt.figure(figsize=(10, 10))
#     plt.bar(x, y[0], label="train score", width=0.3)
#     plt.bar(x + 0.3, y[1], label="test score", width=0.3)
#     plt.xticks(x + 0.15, labels=x_name)
#     plt.title(title)
#     plt.xlabel(xlabel)
#     plt.ylabel(ylabel)
#     plt.legend()
#     plt.savefig(save_dir + "/" + title)
#     plt.clf()
#     plt.close()

In [21]:
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("reg", Ridge(random_state=0))
])

train_score = []
test_score = []

train_MAE = []
test_MAE = []

for turn_num in turn_vari:
    x_train, x_test, y_train, y_test = train_test_split(\
        x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
        y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
        test_size=0.3,
        random_state=0
    )

    pipeline.fit(x_train, y_train)

    train_score.append(pipeline.score(x_train, y_train))
    test_score.append(pipeline.score(x_test, y_test))

    train_predict = pipeline.predict(x_train)
    train_MAE.append(mean_absolute_error(train_predict, y_train))
    test_predict = pipeline.predict(x_test)
    test_MAE.append(mean_absolute_error(test_predict, y_test))

x = [str(i) for i in turn_vari]
y = [train_score, test_score]
two_plot(x, y, "turn number", "score", "standard scaler score", "fig")

y = [train_MAE, test_MAE]
two_plot(x, y, "turn number", "MAE", "standard scaler MAE", "fig")

In [ ]:
pipeline = Pipeline([
    ("scaler", PowerTransformer()),
    ("reg", Ridge(random_state=0))
])

train_score = []
test_score = []

train_MAE = []
test_MAE = []

for turn_num in turn_vari:
    x_train, x_test, y_train, y_test = train_test_split(\
        x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
        y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
        test_size=0.3,
        random_state=0
    )

    pipeline.fit(x_train, y_train)

    train_score.append(pipeline.score(x_train, y_train))
    test_score.append(pipeline.score(x_test, y_test))

    train_predict = pipeline.predict(x_train)
    train_MAE.append(mean_absolute_error(train_predict, y_train))
    test_predict = pipeline.predict(x_test)
    test_MAE.append(mean_absolute_error(test_predict, y_test))

x = [str(i) for i in turn_vari]
y = [train_score, test_score]
two_plot(x, y, "turn number", "score", "power transformed score", "fig")

y = [train_MAE, test_MAE]
two_plot(x, y, "turn number", "MAE", "power transformed MAE", "fig")